<h1 style="text-align:center; color:black; text-decoration:underline red;">Préparation des données pour le projet de cartographie des données validations sur les lignes</h1>

<h2 style="text-decoration:bold; color:black;">Source de données</h2>
<ul>
    <a href="https://data.iledefrance-mobilites.fr/explore/dataset/arrets/export/"><li>Référentiel des arrêts de bus IDFM</li></a>
    <a href="https://data.iledefrance.fr/explore/dataset/geoflar-departements/information/"><li>Contour des départements, GEOFLA</li></a>
    <a href="https://www.insee.fr/fr/statistiques/7739582?sommaire=7728826#consulter"><li>Population légale 2021</li></a>
</ul>

<h2 style="text-decoration:bold; color:black;">Notebook</h2>

In [1]:
import pandas 
import os 
os.environ["USE_PYGEOS"] = "0"
import geopandas

In [2]:
##### PREPARATION DES DONNEES DES COMMUNES #####
com = geopandas.read_file(r"..\DONNEES\communes_idf_2022.shp", encoding="utf-8")
print(com.crs) #Vérifier la projection.
com.to_crs(epsg="3949", inplace=True) #Convertir en mètre.
print(com.crs) #Vérifier la projection.

com.info()

valdoise = com[ com["numdep"] == "95" ]
valdoise.drop(labels=["shape_leng", "fusioinsee", "st_areasha", "st_lengths", "nomcomto"], axis=1, inplace=True)
valdoise.reset_index(drop=True, inplace=True)

save = False 

if save == True :
    valdoise.to_file(filename=r"..\TRAITEMENT\communes_valdoise_2022.geojson", driver="GeoJSON")

valdoise.head()

EPSG:4326
EPSG:3949
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   objectid    1287 non-null   object  
 1   shape_leng  1287 non-null   float64 
 2   insee       1287 non-null   object  
 3   nomcom      1287 non-null   object  
 4   numdep      1287 non-null   object  
 5   fusioinsee  10 non-null     object  
 6   nomcomto    1287 non-null   object  
 7   st_areasha  1287 non-null   float64 
 8   st_lengths  1287 non-null   float64 
 9   geometry    1287 non-null   geometry
dtypes: float64(3), geometry(1), object(6)
memory usage: 100.7+ KB


C:\Users\will\AppData\Local\Temp\ipykernel_1088\3196411111.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valdoise.drop(labels=["shape_leng", "fusioinsee", "st_areasha", "st_lengths", "nomcomto"], axis=1, inplace=True)


,objectid,insee,nomcom,numdep,geometry
0,73,95541,Saint-Clair-sur-Epte,95,"POLYGON ((1603180.203 8220962.671, 1603103.898..."
1,78,95203,Eaubonne,95,"POLYGON ((1648164.534 8197599.814, 1646846.649..."
2,86,95539,Saint-Brice-sous-Forêt,95,"POLYGON ((1653580.606 8198952.869, 1653307.558..."
3,249,95054,Le Bellay-en-Vexin,95,"POLYGON ((1618447.886 8215925.229, 1617885.132..."
4,277,95348,Longuesse,95,"POLYGON ((1620622.799 8205664.615, 1620220.985..."


In [3]:
##### PREPARATION DES DONNEES D'ARRETS #####

#Objectif : obtenir tous les arrêts dans les communes du Val d'Oise.

arrets = geopandas.read_file(filename=r"..\DONNEES\arrets.geojson", geometry="geometry", encoding="utf-8")
#Changer la projection si nécessaire 
def convert_crs(gdf, crs="3949") :
    if gdf.crs != crs :
        print(gdf.crs)
        gdf.to_crs(epsg=crs, inplace=True)
        print(gdf.crs)
    else :
        print(gdf.crs)

    return gdf

arrets = convert_crs(arrets)

#Récupérer les arrêts dans le Val d'Oise 
arrets_vdo = geopandas.sjoin(left_df=arrets, right_df=valdoise, predicate="within")
#arrets_vdo.explore() #Vérifier le résultat.

#Conserver uniquement les colonnes qui nous intéressent :
print(arrets_vdo.columns)
arrets_vdo.drop(labels=['arrcreated', 'arrchanged', 'arrtown',
                        'arrpostalregion', 'arraccessibility', 'arraudiblesignals',
                        'arrvisualsigns', 'arrfarezone', 'index_right'], 
                        axis=1, inplace=True)

arrets_vdo = arrets_vdo[ arrets_vdo["arrtype"] == "bus" ]
arrets_vdo.drop_duplicates(subset="zdaid", inplace=True) #Supprimer les doublons des arrêts.
#Rappel : Les arrêts de bus fonctionnent au minimum par pair.
#Si on conserve tous les arrêts de bus : les plus nombreux seront sur-évalués.

arrets_vdo.head()

if save == True :
    arrets_vdo.to_file(filename=r"..\TRAITEMENT\arrets_bus_vdo.geojson", driver="GeoJSON", encoding="utf-8")

C:\Users\will\AppData\Roaming\Python\Python310\site-packages\geopandas\io\file.py:399: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  as_dt = pd.to_datetime(df[k], errors="ignore")
C:\Users\will\AppData\Roaming\Python\Python310\site-packages\geopandas\io\file.py:399: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  as_dt = pd.to_datetime(df[k], errors="ignore")


EPSG:4326
EPSG:3949
Index(['arrid', 'arrversion', 'arrcreated', 'arrchanged', 'arrname', 'arrtype',
       'arrpubliccode', 'arrxepsg2154', 'arryepsg2154', 'zdaid', 'arrtown',
       'arrpostalregion', 'arraccessibility', 'arraudiblesignals',
       'arrvisualsigns', 'arrfarezone', 'geometry', 'index_right', 'objectid',
       'insee', 'nomcom', 'numdep'],
      dtype='object')


In [5]:
##### PREPARER LES DONNEES DEMOGRAPHIQUES #####

pop21 = pandas.read_csv(r"..\DONNEES\donnees_communes.csv", sep=";", decimal=",", encoding="utf-8")

pop_vdo = pop21[ pop21["DEP"] == "95" ]
pop_vdo.reset_index(drop=True, inplace=True)

pop_vdo["COM"] = pop_vdo["COM"].apply(func=lambda x : str(x))

if save == True :
    pop_vdo.to_csv(r"..\TRAITEMENT\population_2021.csv", 
               sep=",", decimal=".", encoding="utf-8")

pop_vdo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   REG      184 non-null    int64 
 1   Région   184 non-null    object
 2   DEP      184 non-null    object
 3   CODARR   184 non-null    int64 
 4   CODCAN   184 non-null    object
 5   CODCOM   184 non-null    int64 
 6   COM      184 non-null    object
 7   Commune  184 non-null    object
 8   PMUN     184 non-null    int64 
 9   PCAP     184 non-null    int64 
 10  PTOT     184 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 15.9+ KB


C:\Users\will\AppData\Local\Temp\ipykernel_1088\373795713.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_vdo["COM"] = pop_vdo["COM"].apply(func=lambda x : str(x))
